In [11]:
input_string = "AZAFH DXKEB EEGEL LFMKZ ZQEKV LQIIY AXNHB JFPGP FQBXL FQQZO LKNHB JTMLH LNIVR SEMGZ WANVV EBTXA WTMEW DQALU WEAAH VPMLJ WZLXK MBWGO AYBHI WSQGD AFPAL VULGV LWVHD OUBAH FKKXY LMQGA QFPTA LTQLD SEVBU WFMXU WUOAA QRWNY AFUNZ LNMKV MZLTI GGBMO SFLTA WEQGJ WTMPH KRIBY DKANY WFPTA ZUATN WIILA ZUZMF FUVXH FPPXI WXQXC WPBAH LTMAH VNMXU TAZGP FZQGL LQMGM GDBRM GGZHY FUVXA WQVYV JFGYP NQJNA AFETZ FQDXY HAALP TXMGV OMLTF KFWIP FPWPU SZGWH LQEBA ZUVTF WMZHY LIW".replace(" ","")
string_length = len(input_string)

Find substrings of length 3-5 with its occurence number

In [37]:
candidates = {}
for substring_length in range(3,6):
    for i in range(string_length-substring_length+1):
        current_substring = input_string[i:i+substring_length]
        if current_substring not in candidates:
            candidates[current_substring] = input_string.count(current_substring)

Get the 4 values that repeat most often

In [42]:
sorted(candidates.items(),key=lambda x:x[1],reverse=True)[0:4]

[('AZU', 3), ('NHB', 2), ('HBJ', 2), ('GPF', 2)]

Start with "AZU"

In [56]:
s = "AZU"
current_string = input_string
global_index=0
for i in range(3):
    index =  current_string.find(s)
    global_index=global_index+3+index
    print(global_index)
    current_string = current_string[index+3:]

232
242
367


The differences are 10 and 125. gcd(10,125)=5 and is a candidate for the length of the keyword. To be sure, lets verify some more candidates:

In [59]:
s = "NHB"
current_string = input_string
global_index=0
for i in range(3):
    index =  current_string.find(s)
    global_index=global_index+3+index
    print(global_index)
    current_string = current_string[index+3:]

35
55
57


For "NHB" the candidates are 20 and 2. 20 works with running hypothesis, 2 doesnt.

In [60]:
s = "HBJ"
current_string = input_string
global_index=0
for i in range(3):
    index =  current_string.find(s)
    global_index=global_index+3+index
    print(global_index)
    current_string = current_string[index+3:]

36
56
58


For "HBJ" the candidates are 20 and 2. 20 works with running hypothesis, 2 doesnt.
Let's assume the key is 5 long, otherwise we can come back to the 2.

In [62]:
keylength = 5

For each key-element, create a list of occurences of each letter.

In [85]:
list=[]
for offset in range(keylength):
    occurences = {chr(j): 0 for j in range(ord('A'), ord('Z')+1)}
    for letter in range(offset, string_length, 5):
        occurences[input_string[letter]]+=1
    list.append(occurences)

For each key-element print the two most often appearing letters

In [125]:
freq_list=[]
for i in list:
    freq_list.append(sorted(i.items(),key=lambda x:x[1],reverse=True)[0:2])
    print(sorted(i.items(),key=lambda x:x[1],reverse=True)[0:2])

[('W', 15), ('L', 12)]
[('F', 11), ('Q', 10)]
[('M', 12), ('Q', 7)]
[('G', 11), ('X', 11)]
[('A', 10), ('H', 9)]


Two most appearing letters in German and English language:
Ger : E-17%, N-10%
Eng : E-11%, A-8,5%

Observe G and X have the same probability

Starting with key-element 0, assume W (alternative L) is decrypted to E:

In [155]:
key = []
alternative = []
for i in range(5):
    diff=(ord(freq_list[i][0][0]) - ord('E'))%26
    key.append(chr(ord('A') + diff))
    diff=(ord(freq_list[i][1][0]) - ord('E'))%26
    alternative.append(chr(ord('A') + diff))

In [156]:
print(key)
print(alternative)

['S', 'B', 'I', 'C', 'W']
['H', 'M', 'M', 'T', 'D']


In [157]:
key_candidates = [[] for i in range(5)]
for i in range(5):
    key_candidates[i].append(key[i])
    key_candidates[i].append(alternative[i])

[['S', 'H'], [], [], [], []]
[['S', 'H'], ['B', 'M'], [], [], []]
[['S', 'H'], ['B', 'M'], ['I', 'M'], [], []]
[['S', 'H'], ['B', 'M'], ['I', 'M'], ['C', 'T'], []]
[['S', 'H'], ['B', 'M'], ['I', 'M'], ['C', 'T'], ['W', 'D']]


Starting with key-element 0, assume W (alternative L) is decrypted to A:

In [158]:
key = []
alternative = []
for i in range(5):
    diff=(ord(freq_list[i][0][0]) - ord('A'))%26
    key.append(chr(ord('A') + diff))
    diff=(ord(freq_list[i][1][0]) - ord('A'))%26
    alternative.append(chr(ord('A') + diff))

In [159]:
print(key)
print(alternative)
for i in range(5):
    key_candidates[i].append(key[i])
    key_candidates[i].append(alternative[i])

['W', 'F', 'M', 'G', 'A']
['L', 'Q', 'Q', 'X', 'H']


In [160]:
key_candidates

[['S', 'H', 'W', 'L'],
 ['B', 'M', 'F', 'Q'],
 ['I', 'M', 'M', 'Q'],
 ['C', 'T', 'G', 'X'],
 ['W', 'D', 'A', 'H']]

In [201]:
def decrypt(key, ciphertext = input_string):
    key_length= len(key)
    plaintext = ""
    for i in range(len(ciphertext)):
        key_letter = key[i%key_length]
        diff=(ord(ciphertext[i]) - ord(key_letter))%26
        plaintext+=(chr(ord('A') + diff))
    return plaintext

We want to test all possible keys till a valid plaintext comes out, lets start with the most possible one - SBICW and SBITW:

In [205]:
print(decrypt("SBICW"))
print(decrypt("SBItW"))

IYSDLLWCCFMDYCPTEEIDHPWIZTPAGCIWFFFREHETNPTVPNPIXSTJFFFRSEJLTMATVADEEDEZFTZMALVEESECALPSJYEDSYLDOEJNEYDVOUAOESIXTFMERIEHIEHYPDTDEZTVNFHWTTYLNJCVCTLIEEYEHRETSIJHADNZYEEEVYETGYEYQOLCIEMLDTMEIZUYDRMOFTKSAEDREEDIENESENLSQAZCLJSLCEEHREHTSRREHAJEHTRKJNTNVLNOHVMEWIVGEOTYLTSEYLDMEVYBZRETNYIEPTPEEQOCTPQOFRFCNTNVEEPNWZREYWTVPBLEIEWRDNPVVCPZSJTBWEEZWLDRJSEOGTNOONYAYYULTPWZEHTNRJELRFCTHO
IYSGLLWCFFMDYFPTEELDHPWLZTPAJCIWFIFREHHTNPTYPNPIASTJFIFRSEMLTMAWVADEHDEZFWZMALYEESEFALPSMYEDSBLDOEMNEYDYOUAOHSIXTIMERIHHIEHBPDTDHZTVNIHWTTBLNJCYCTLIHEYEHUETSIMHADNCYEEEYYETGBEYQOOCIEMODTMELZUYDUMOFTNSAEDUEEDIHNESEQLSQACCLJSOCEEHUEHTSUREHAMEHTRNJNTNYLNOHYMEWIYGEOTBLTSEBLDMEYYBZRHTNYIHPTPEHQOCTSQOFRICNTNYEEPNZZREYZTVPBOEIEWUDNPVYCPZSMTBWEHZWLDUJSEOJTNOOQYAYYXLTPWCEHTNUJELRICTHO


Looks strange, lets keep on going:

In [229]:
for i in range(4**5):
    string_candidate=""
    for j in range(5):
        string_candidate+=key_candidates[j][round(i/4**j)%4]
    if string_candidate.startswith("SMIT"):
        print(decrypt(string_candidate))

INSMLLLCLFMSYLPTTERDHEWRZTEAPCILFOFRTHNTNETEPNEIGSTYFOFRHESLTBACVASENDEOFCZMPLEEEHELALESSYESSHLDDESNENDEOUPONSIMTOMEGINHITHHPDIDNZTKNOHWITHLNYCECTAINEYTHAETHISHASNIYETEEYEIGHEYFOUCITMUDTBERZUNDAMOUTTSATDAEESINNEHEWLSFAICLYSUCETHAEHISAREWASEHIRTJNINELNDHEMELIEGEDTHLTHEHLDBEEYBORNTNNINPTEENQORTYQOUROCNINEEEENFZRTYFTVEBUEITWADNEVECPOSSTBLENZWADAJSTOPTNDOWYANYDLTEWIEHINAJEAROCTWO
INSMELLCLYMSYLITTERWHEWRSTEAPVILFOYRTHNMNETEINEIGLTYFOYRHESETBACOASENWEOFCSMPLEXEHELTLESSRESSHEDDESGENDEHUPONLIMTOFEGINAITHHIDIDNSTKNOAWITHENYCEVTAINXYTHAXTHISAASNIRETEEREIGHXYFOUVITMUWTBERSUNDAFOUTTLATDAXESINGEHEWESFAIVLYSUVETHAXHISAKEWASXHIRTCNINEENDHEFELIEZEDTHETHEHEDBEERBORNMNNINITEENJORTYJOUROVNINEXEENFSRTYFMVEBUXITWAWNEVEVPOSSMBLENSWADACSTOPMNDOWRANYDETEWIXHINACEAROVTWO
INSMHLLCLBMSYLLTTERZHEWRVTEAPYILFOBRTHNPNETELNEIGOTYFOBRHESHTBACRASENZEOFCVMPLEAEHELWLESSUESSHHDDESJENDEKUPONOIMTOIEGINDITHHLDIDNVTKNODWITHHNYCEYTAINAYTHAATHISDASNIUETEEUEIGHAYFOUYITMUZTBERVUNDAIOUTTOATDAAESINJEHEWHSFAIYLYSUYETHAAHISANEWASAHI